### Code initialisation

In [1]:
import os
from pathlib import Path
import matplotlib as mpl

from my_functions_sp_scaled import *

# Define whether to filter out values <0.1
remove_little_values = True
model_directory = '../../../../FloodModelling/Model_SinglePeak_Scaled/'

In [2]:
methods = ['6h_feh_sp','6h_sp_+0','6h_sp_+5','6h_sp_+10','6h_sp_+20']  

In [3]:
fps = []
for method_num, method in enumerate(methods):
    if method == "6h_feh_sp":
        model_directory = '../../../../FloodModelling/Model_FEH_profiles/'
    else:
        model_directory = '../../../../FloodModelling/Model_SinglePeak_Scaled/'
    
    fp = model_directory + "{}/{} (Max).Resampled.Terrain.tif".format(method, '{}')
    fps.append(fp)

In [4]:
breaks_difference = np.array([-0.1, 0.1, 0.3, 100])

#### Read in rasters containing depths and extents using each of the methods, and store these in a dictionary
#### Create versions of depth/velocity rasters which are classified into depth/velocity categories

In [173]:
# from osgeo import gdal
# raster = gdal.Open(fp.format('Depth'))
# gt =raster.GetGeoTransform()
# pixelSizeX = gt[1]
# pixelSizeY =-gt[5]
# pixelSizeX, pixelSizeY

In [5]:
def do_processing (variable_name):    
    
    ######################################################################################
    # Specify breaks and labels for this variable
    ######################################################################################
    if variable_name == 'Depth':
        breaks = np.array([0, 0.3, 0.6, 1.2, 100])  
        labels = ['<=0.3m', '0.3-0.6m', '0.6-1.2m', '>1.2m']
        classified_colors_list = [mpl.cm.Blues(0.2), mpl.cm.Blues(0.5), mpl.cm.Blues(0.7),"navy"]
    elif variable_name == 'Velocity':
        breaks = np.array([0,0.25,0.5,2,100])
        labels = ["<=0.25m/s", "0.25-0.5m/s", "0.5-2m/s", ">2m/s"]
        classified_colors_list = [mpl.cm.cool(0.3), mpl.cm.cool(0.5), mpl.cm.cool(0.7), mpl.cm.cool(0.9)]
        
    ######################################################################################
    # For the single peak scenarios:
        # Classify depth/velocity rasters into depth/velocity bins
        # Save as raster
        # Plot this raster, and save the output
    ######################################################################################
    single_peak, out_meta = prepare_rainfall_scenario_raster(fps[0].format(variable_name), remove_little_values)
    singlepeak_classified = classify_raster(single_peak, breaks)
    fp_for_classified_raster = fps[0].format(variable_name).replace('{} (Max).Resampled.Terrain'.format(variable_name),'{}_classified'.format(variable_name))
    save_array_as_raster(singlepeak_classified, fp_for_classified_raster, out_meta) 
    plot_classified_raster(fp_for_classified_raster, labels, classified_colors_list)
    
    ######################################################################################
    # For the other rainfall scenarios:
        # Classify depth/velocity rasters into depth/velocity bins
            # Save as raster
            # Plot this raster, and save the output
        # Find the difference between depth/velocity rasters for this scenario and the single peak scenario:
            # Save this as a raster
            # Plot this raster and save the output
        # Reclassify the difference rasters into bins:
            # Save this as a raster
            # Plot this raster and save the output
        # Reclassify the difference rasters into positive, negative or neutral differences:
            # Save this as a raster
            # Plot this raster and save the output
    ######################################################################################
    for fp in fps[1:]:
        # Create directory to store results
        # processing_dir = "HecRas_Processing/" + fp.split('/')[1]
        # Path(processing_dir).mkdir(parents=True, exist_ok=True)
    
        # Classify depth/velocity rasters into depth/velocity bins
        this_scenario_raster = prepare_rainfall_scenario_raster(fp.format(variable_name), remove_little_values)[0]
        this_scenario_classified = classify_raster(this_scenario_raster, breaks)
        fp_for_classified_raster = fp.replace('{} (Max).Resampled.Terrain','{}_classified'.format(variable_name) )
        save_array_as_raster(this_scenario_classified, fp_for_classified_raster, out_meta) 
        plot_classified_raster(fp_for_classified_raster, labels, classified_colors_list)    

        # Find difference between depth/velocity rasters from this method, and the single peak method
        difference_raster = single_peak - this_scenario_raster
        # fp_for_difference_raster = "Arcpy/{}_singlepeak_{}_diff.tif".format(variable_name, rainfall_scenario_name)
        # save_array_as_raster(singlepeak_classified, fp_for_difference_raster, out_meta) 
        # plot_difference(variable_name, rainfall_scenario_name, labels, 'YlGnBu') 

        # Reclassify the difference rasters to categories 
        classified_difference_raster = classify_raster (difference_raster, breaks_difference)
        fp_for_classified_diff_raster = fp.replace('{} (Max).Resampled.Terrain','{}_difffromsinglepeak_classified'.format(variable_name))
        save_array_as_raster(classified_difference_raster, fp_for_classified_diff_raster, out_meta) 
        plot_difference_levels(fp_for_classified_diff_raster, labels)    

        # Reclassify the difference rasters to represent whether value is positive or negative 
        pos_neg_depth_diff = np.where((np.isnan(difference_raster)), np.nan, 
                                  np.where((difference_raster > 0), 0,
                                    np.where((difference_raster == 0), 1,
                                        np.where((difference_raster < 0), 2, 500))))
        fp_for_posneg_diff_raster = fp.replace('{} (Max).Resampled.Terrain','{}_difffromsinglepeak_posneg'.format(variable_name) )
        save_array_as_raster(pos_neg_depth_diff, fp_for_posneg_diff_raster, out_meta) 
        plot_difference_levels_pos_neg(fp_for_posneg_diff_raster)  

        
# Run the function  
do_processing('Depth')
do_processing('Velocity')

KeyboardInterrupt: 

# Hazard calculations
The hazard is calculated based on definition here: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/842485/What-is-the-Risk-of-Flooding-from-Surface-Water-Map.pdf . 

### Calculate the hazard categories for the single peak results

In [12]:
# Read in depth and velocity rasters
sp_depth, out_meta = prepare_rainfall_scenario_raster(fps[0].format('Depth'), remove_little_values)
sp_velocity = prepare_rainfall_scenario_raster(fps[0].format('Velocity'), remove_little_values)[0]

# Create a composite hazard array
sp_hazard= np.where((sp_depth.flatten())<0.25, (sp_depth.flatten()* (sp_velocity.flatten()+0.5) + 0.5) , (sp_depth.flatten() * (sp_velocity.flatten()+0.5) + 1))
sp_hazard = sp_hazard.reshape(sp_depth.shape)

# Classify this according to bins
breaks_hazard = np.array([0, 0.5, 0.75, 1.25,2, 100])  
sp_classified_hazard = classify_raster(sp_hazard, breaks_hazard)
# Save to file
fp_for_hazard = fps[0].replace('{} (Max).Resampled.Terrain','{}_classified'.format('hazard'))
save_array_as_raster(sp_classified_hazard, fp_for_hazard, out_meta) 

# Plot the classified hazard categories
labels_hazard = ['Low hazard', 'Moderate hazard', 'Significant hazard', 'Extreme hazard']
classified_colors_list_hazard = [mpl.cm.autumn_r(0.2),mpl.cm.autumn_r(0.5), mpl.cm.autumn_r(0.7),"darkred"]
plot_classified_raster(fp_for_hazard, labels_hazard, classified_colors_list_hazard) 

### Calculate the hazard categories for the other scenarios, and then find the difference in the categories between those and single peak

In [15]:
for fp in fps[1:]:

    # Read in depth and velocity rasters
    depth = prepare_rainfall_scenario_raster(fp.format('Depth'), remove_little_values)[0]
    velocity = prepare_rainfall_scenario_raster(fp.format('Velocity'), remove_little_values)[0]
    
    # Create a composite hazard array
    hazard= np.where((depth.flatten())<0.25, (depth.flatten()* (velocity.flatten()+0.5) + 0.5) , (depth.flatten() * (velocity.flatten()+0.5) + 1))
    hazard = hazard.reshape(depth.shape)
    
    # Classify this according to bins
    classified_hazard = classify_raster(hazard, breaks_hazard)
    # Save to file
    fp_for_hazard = fp.replace('{} (Max).Resampled.Terrain','{}_classified'.format('hazard'))
    save_array_as_raster(classified_hazard, fp_for_hazard, out_meta) 
    # Plot the classified hazard categories
    labels_hazard = ['Low hazard', 'Moderate hazard', 'Significant hazard', 'Extreme hazard']
    classified_colors_list_hazard = [mpl.cm.autumn_r(0.2),mpl.cm.autumn_r(0.5), mpl.cm.autumn_r(0.7),"darkred"]
    plot_classified_raster(fp_for_hazard, labels_hazard, classified_colors_list_hazard) 
    
    # Find the difference in category between this scenario and the single peak scenario 
    difference_raster = sp_classified_hazard - classified_hazard
    # abs_difference_raster =abs(difference_raster)
    
    # Create correct list of labels
    unique_vals = pd.DataFrame({'Unique_val':np.unique(difference_raster)})
    replacement_dict = {-3.0: 'Hazard_3CatsLower', -2.0 : 'Hazard_2CatsLower', -1.0 : 'Hazard_1CatsLower', 0: 'Hazard_SameCat',
        1 : 'Hazard_1CatsHigher', 2: 'Hazard_2CatsHigher', 3: 'Hazard_3CatsHigher'}
    unique_vals=unique_vals.replace({"Unique_val": replacement_dict})
    labels = unique_vals['Unique_val'][:-1]
    labels_hazard_diff = labels.tolist()
    
    # Plot the difference
    fp_for_diff_raster = fp.replace('{} (Max).Resampled.Terrain','hazard_cat_difference')
    save_array_as_raster(difference_raster, fp_for_diff_raster, out_meta) 
    plot_diff_hazard_cats(fp_for_diff_raster, labels_hazard_diff, classified_colors_list_hazard)